In [ ]:
import numpy as np
import copy
import pandas as pd
from openpyxl.workbook import Workbook
import matplotlib.pyplot as plt

current = -1
def RoundRobin(queues):
    global current
    current = (current+1)%num_queues
    return current

def ShortestFirst(queues):
    return queues.index(min(queues))

def ChooseQueue(queues):
    return ShortestFirst()

# def printDebug(chooseQueue, queue):
#     queue_name = ""
#     if chooseQueue == RoundRobin:
#         queue_name = "RR"
#     if chooseQueue == ShortestFirst:
#         queue_name = "SF"
#     print(queue_name, ' queue: ', queue)

def writeDataToExcel(data, trial_number):
#     flat_data = []

#     # Create a DataFrame
#     df = pd.DataFrame(data)

#     # Specify the path where you want to save the Excel file
#     excel_file_path = 'output.xlsx'

#     # Write the DataFrame to an Excel file
#     df.to_excel(excel_file_path, index_label='Index')

#     print(f"Data has been exported to {excel_file_path}")

    # Initialize empty DataFrames for RR and SF
    rr_df_rows = []
    sf_df_rows = []

    # Iterate through the list and populate the DataFrames
    for entry in data:
        minute = entry['minute']

        # RR_info
        rr_row = {
            'minute': minute,
            'queues': entry['RoundRobin']['Queues'],
            'arrivals': entry['RoundRobin']['Arrivals'],
            'departures': entry['RoundRobin']['Departures'],
            'delta': entry['RoundRobin']['Delta']
        }
        rr_df_rows.append(rr_row)

        # SF_info
        sf_row = {
            'minute': minute,
            'queues': entry['ShortestFirst']['Queues'],
            'arrivals': entry['ShortestFirst']['Arrivals'],
            'departures': entry['ShortestFirst']['Departures'],
            'delta': entry['ShortestFirst']['Delta']
        }
        sf_df_rows.append(sf_row)
        
    rr_df = pd.DataFrame(rr_df_rows)
    sf_df = pd.DataFrame(sf_df_rows)

    # Save DataFrames to Excel
    with pd.ExcelWriter('output.xlsx',engine='openpyxl' ,mode='a') as writer:
        rr_df.to_excel(writer, sheet_name='RoundRobin' + trial_number, index=False)
        sf_df.to_excel(writer, sheet_name='ShortestFirst' + trial_number, index=False)


def simulate(num_queues,
            simulation_time,
            average_arrival_rate_main,
            average_departure_rate_total,
            printFlag = False):

    # Initialize queues with empty queue sizes
    RR_queues = [0] * num_queues
    SF_queues = [0] * num_queues
    
#     print('RR initial queue: ', RR_queues)
#     print('SF initial queue: ', SF_queues)
        
    total_time_minutes = 0  # Initialize total time
    
    info = []

    def GetArrivalsToMainQueueThisMinute():
        return np.random.poisson(average_arrival_rate_main)

    def DivideArrivals(total, queues, chooseQueue):
#         printDebug(chooseQueue, queues)
        arrivals_this_minute = [0] * num_queues
        for i in range(total):
            queue_index = chooseQueue(queues)
            arrivals_this_minute[queue_index] += 1
            queues[queue_index] += 1
        return arrivals_this_minute

    def DepartFromQueues(queues):
        departures_this_minute = [0] * num_queues
        # Check if each queue is not empty before generating departures
        for i in range(num_queues):
            if queues[i] > 0:
                departures_this_minute[i] = min(np.random.poisson(average_departure_rate_total / num_queues), queues[i])
                queues[i] -= departures_this_minute[i]
        return departures_this_minute

    for i in range(simulation_time):

        arrivals_main_this_minute = GetArrivalsToMainQueueThisMinute()
        arrivals_this_minute_RR = DivideArrivals(arrivals_main_this_minute, RR_queues, RoundRobin)
        arrivals_this_minute_SF = DivideArrivals(arrivals_main_this_minute, SF_queues, ShortestFirst)

        departures_this_minute_RR = DepartFromQueues(RR_queues)
        departures_this_minute_SF = DepartFromQueues(SF_queues)

        delta_RR = max(RR_queues) - min(RR_queues)
        delta_SF = max(SF_queues) - min(SF_queues)

#         print('RR queue main: ', RR_queues)
#         print('SF queue main: ', SF_queues)
            
        RR_info = {
            "Queues": RR_queues,
            "Arrivals" : arrivals_this_minute_RR,
            "Departures" : departures_this_minute_RR,
            "Delta" : delta_RR
                  }

        SF_info = {
            "Queues": SF_queues,
            "Arrivals" : arrivals_this_minute_SF,
            "Departures" : departures_this_minute_SF,
            "Delta" : delta_SF
                  }

        minute_info = {
            "minute" : i+1,
            "RoundRobin" : copy.deepcopy(RR_info),
            "ShortestFirst" : copy.deepcopy(SF_info)
                      }
        info.append(minute_info)

    return info;

num_queues = 10  # Number of queues
simulation_time = 100
average_arrival_rate_main = 0.8*num_queues  # arrivals rate per minute at the main queue
average_departure_rate_total = 1  # departures rate per minute from a single queue
data = []
        
for i in range (10):
    info = simulate(num_queues,
            simulation_time,
            average_arrival_rate_main,
            average_departure_rate_total)
    
    data.append(copy.deepcopy(info))
#     for min_info in info:
#         print(f"min ", min_info["minute"])
#         print(f"RR ", min_info["RoundRobin"])
#         print(f"SF ", min_info["ShortestFirst"])
#     writeDataToExcel(info, str(i+1))
    

In [ ]:
# import numpy as np
# import copy
# import pandas as pd
# from openpyxl.workbook import Workbook
# import matplotlib.pyplot as plt
# import json

# # read by default 1st sheet of an excel file
# sr_delta_sum = [0] * 100
# rr_delta_sum = [0] * 100
# difflist = []
# for i in range(10): 
#     sr_sheetname = 'ShortestFirst' + str(i+1)
#     rr_sheetname = 'RoundRobin' + str(i+1)

#     sr_data = pd.read_excel('output.xlsx',sr_sheetname)
#     rr_data = pd.read_excel('output.xlsx',rr_sheetname)

#     x_values = range(100)
#     ShortestFirst = sr_data['queues']
#     RR_values = rr_data['queues'] 
# #     SS_queues = ast.literal_eval(ShortestFirst)
# #     RR_queues = ast.literal_eval(RR_values)
#     # SF_values = [data[0]['ShortestFirst']["Delta"]]
#     sf, rr = sum(json.loads(ShortestFirst[99])), sum(json.loads(RR_values[99]))
#     print("simulation number: ", i+1)
#     print("SF: ", sf)
#     print("RR: ", rr)
#     print("diff: ", rr-sf)
#     difflist.append(rr-sf)
# print(sum(difflist) / len(difflist))